## Read me

1. ``pip install tensorflow``

2. Put data files in
``cancer-net/pnet/_database``
which can be downloaded from Google Drive [here](https://drive.google.com/uc?id=17nssbdUylkyQY1ebtxsIw5UzTAd0zxWb&export=download).

3. To install pysr, use ``conda install -c conda-forge pysr``, then run ``python -c 'import pysr; pysr.install()'``



In [1]:
import os
import importlib
import tensorflow as tf
import pandas as pd
import numpy as np

import pysr

from cancernet.pnet.config_path import PROSTATE_LOG_PATH, POSTATE_PARAMS_PATH
from cancernet.pnet.model.model_factory import get_model
from cancernet.pnet.model.nn import Model
from cancernet.pnet.model.builders.builders_utils import get_map_from_layer, get_layer_maps

from cancernet.pnet.data.data_access import Data
from cancernet.pnet.data.prostate_paper.data_reader import ProstateDataPaper, load_data_type
from cancernet.pnet.data.prostate_paper import data_reader
from cancernet.pnet.data.pathways.reactome import ReactomeNetwork


from sklearn.metrics import roc_auc_score, roc_curve, auc, average_precision_score, f1_score, accuracy_score, \
    precision_score, recall_score

2022-07-15 16:33:33.915452: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-15 16:33:33.918832: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/sw/nix/store/f0ycdncw8dw4wlicnzm74lgv9c51rlg4-openblas-0.3.20/lib:/cm/shared/apps/slurm/current/lib64
2022-07-15 16:33:33.918843: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
params_file = '.onsplit_average_reg_10_tanh_large_testing'
params_file_full = "cancernet.pnet.train.params.P1000.pnet" + params_file
params = importlib.import_module(params_file_full)

In [3]:
# manually adjusted P-net model params
n_hidden_layers = 2
params.models[0]['params']['model_params']['n_hidden_layers'] = n_hidden_layers
params.models[0]['params']['model_params']['w_reg'] = [0.001] * (n_hidden_layers+1)
params.models[0]['params']['model_params']['w_reg_outcomes'] = [0.001] * (n_hidden_layers+1)
params.models[0]['params']['model_params']['dropout'] = [0.5]+[0.1] * (n_hidden_layers+1)
params.models[0]['params']['model_params']['loss_weights'] = [2] * (n_hidden_layers-6) + [2, 7, 20, 54, 148, 400]

In [4]:
# model inherits a sklearn BaseEstimator
model = Model(**params.models[0]['params'])
model.get_params()

{'model_params': {'use_bias': True,
  'w_reg': [0.001, 0.001, 0.001],
  'w_reg_outcomes': [0.001, 0.001, 0.001],
  'dropout': [0.5, 0.1, 0.1, 0.1],
  'loss_weights': [2, 7, 20, 54, 148, 400],
  'optimizer': 'Adam',
  'activation': 'tanh',
  'data_params': {'id': 'ALL',
   'type': 'prostate_paper',
   'params': {'data_type': ['mut_important', 'cnv_del', 'cnv_amp'],
    'drop_AR': False,
    'cnv_levels': 3,
    'mut_binary': True,
    'balanced_data': False,
    'combine_type': 'union',
    'use_coding_genes_only': True,
    'selected_genes': 'tcga_prostate_expressed_genes_and_cancer_genes.csv',
    'training_split': 0}},
  'add_unk_genes': False,
  'shuffle_genes': False,
  'kernel_initializer': 'lecun_uniform',
  'n_hidden_layers': 2,
  'attention': False,
  'dropout_testing': False},
 'fitting_params': {'samples_per_epoch': 10,
  'select_best_model': False,
  'monitor': 'val_o6_f1',
  'verbose': 2,
  'epoch': 300,
  'shuffle': True,
  'batch_size': 50,
  'save_name': 'pnet',
  'debug

In [5]:
ret = model.build_fn(**model.model_params)
pnet_mod, feature_names, reactome_map = ret

{'id': 'ALL', 'type': 'prostate_paper', 'params': {'data_type': ['mut_important', 'cnv_del', 'cnv_amp'], 'drop_AR': False, 'cnv_levels': 3, 'mut_binary': True, 'balanced_data': False, 'combine_type': 'union', 'use_coding_genes_only': True, 'selected_genes': 'tcga_prostate_expressed_genes_and_cancer_genes.csv', 'training_split': 0}}
n_hidden_layers 2


/mnt/home/ttesileanu/Documents/research/deep-cancer-conda/cancernet/pnet/data/prostate_paper/data_reader.py:69: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x = x.loc[:, intersect]
/mnt/home/ttesileanu/Documents/research/deep-cancer-conda/cancernet/pnet/data/prostate_paper/data_reader.py:69: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x = x.loc[:, intersect]
/mnt/home/ttesileanu/Documents/research/deep-cancer-conda/cancernet/pnet/data/prostate_paper/data_reader.py:69: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x = x.loc[:, intersect]


(1011, 27687)
(1011, 1)
(1011,)
(27687,)
input dimension 27687 self.units 9229
n_inputs_per_node 3
self.kernel_initializer <keras.regularizers.L2 object at 0x7fcb6c1cbc10> <keras.initializers.initializers_v2.LecunUniform object at 0x7fcb6c1cb370> <keras.regularizers.L2 object at 0x7fcb6c1cbc10>
input dimensions (None, 27687)


2022-07-15 16:33:47.265391: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/sw/nix/store/f0ycdncw8dw4wlicnzm74lgv9c51rlg4-openblas-0.3.20/lib:/cm/shared/apps/slurm/current/lib64
2022-07-15 16:33:47.265415: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-07-15 16:33:47.265709: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX51

pathways 146
genes 10621
pathways 26
genes 147
pathways 1
genes 26
original dropout [0.5, 0.1, 0.1, 0.1]
dropout [1, 2] [0.5, 0.1, 0.1, 0.1] [0.001, 0.001, 0.001]
layer 0, dropout  0.1 w_reg 0.001
layer 1, dropout  0.1 w_reg 0.001
Compiling...
loss_weights [2, 7, 20, 54, 148, 400]
[<keras.engine.input_layer.InputLayer object at 0x7fccb00ceb20>, <cancernet.pnet.model.layers_custom.Diagonal object at 0x7fcb6c1cbbb0>, <keras.layers.regularization.dropout.Dropout object at 0x7fcb6bfc4670>, <cancernet.pnet.model.layers_custom.SparseTF object at 0x7fcb685a95e0>, <keras.layers.regularization.dropout.Dropout object at 0x7fcb6859c190>, <cancernet.pnet.model.layers_custom.SparseTF object at 0x7fcb6be156a0>, <keras.layers.core.dense.Dense object at 0x7fcb685ac940>, <keras.layers.core.dense.Dense object at 0x7fcb685a93a0>, <keras.layers.core.dense.Dense object at 0x7fcb67a33040>, <keras.layers.core.activation.Activation object at 0x7fcb685ace50>, <keras.layers.core.activation.Activation object at 

In [6]:
tf.keras.utils.plot_model(pnet_mod)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [7]:
data = Data(**params.data[0])

x_train, x_validate_, x_test_, y_train, y_validate_, y_test_, info_train, info_validate_, info_test_, cols = data.get_train_validate_test()

print(x_train.shape)

/mnt/home/ttesileanu/Documents/research/deep-cancer-conda/cancernet/pnet/data/prostate_paper/data_reader.py:69: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x = x.loc[:, intersect]
/mnt/home/ttesileanu/Documents/research/deep-cancer-conda/cancernet/pnet/data/prostate_paper/data_reader.py:69: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x = x.loc[:, intersect]
/mnt/home/ttesileanu/Documents/research/deep-cancer-conda/cancernet/pnet/data/prostate_paper/data_reader.py:69: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x = x.loc[:, intersect]


(807, 27687)


In [8]:
pnet_mod.fit(
    x_train, y_train,
    validation_data=(x_validate_, y_validate_),
    epochs=20,
    batch_size=10,
    verbose=0
)

input dimensions (None, 27687)
input dimensions (None, 27687)
input dimensions (None, 27687)


In [9]:
y_hat = pnet_mod.predict(x_test_)
print([y.shape for y in y_hat])

input dimensions (None, 27687)
4/4 [==============================] - 0s 5ms/step
[(102, 1), (102, 1), (102, 1)]


In [10]:
ys = y_test_.flatten()
outs = y_hat[-1]

fpr_valid, tpr_valid, _ = roc_curve(ys, outs)
test_auc = auc(fpr_valid, tpr_valid)

print('accuracy', accuracy_score(ys, outs>0.5))
print('auc', test_auc)
print('aupr', average_precision_score(ys, outs))
print('f1', f1_score(ys, outs>0.5))
print('precision', precision_score(ys, outs>0.5))
print('recall', recall_score(ys, outs>0.5))

accuracy 0.8921568627450981
auc 0.9515570934256055
aupr 0.9166715814427417
f1 0.8307692307692308
precision 0.8709677419354839
recall 0.7941176470588235


In [11]:
layer_dict = {x.name:x for x in pnet_mod.layers}
explain_mod = tf.keras.models.Model(
    inputs=pnet_mod.inputs,
    outputs=[layer_dict['h0'].output, layer_dict['h1'].output, layer_dict['h2'].output, layer_dict['o_linear3'].output]
)

In [12]:
reactome = ReactomeNetwork()
pathway_names = reactome.reactome.pathway_names.copy()
pathway_names.index = pathway_names['reactome_id']
pathway_names['pathway_name'] = pathway_names['pathway_name'].str.replace(' ','_').str.replace('-','_')

In [13]:
data_for_pysr = explain_mod.predict(x_train)
print(len(data_for_pysr))

input dimensions (None, 27687)
26/26 [==============================] - 0s 4ms/step
4


In [14]:
h0 = pd.DataFrame(data_for_pysr[0], columns=reactome_map[0].index)
h1 = pd.DataFrame(data_for_pysr[1], columns=pathway_names.loc[reactome_map[1].index]['pathway_name'])
h2 = pd.DataFrame(data_for_pysr[2], columns=pathway_names.loc[reactome_map[2].index]['pathway_name'])
out = pd.DataFrame(data_for_pysr[3], columns=['out'])

In [15]:
h1 = (h1-h1.mean())/(h1.std())

In [16]:
tmp = h1.join(out)
tmp_corr = tmp.corr()['out'].abs().sort_values(ascending=False).drop('out')
tmp_corr.head(20)

Post_translational_protein_modification         0.951626
RNA_Polymerase_II_Transcription                 0.931611
Cellular_responses_to_stress                    0.816716
Signaling_by_Rho_GTPases                        0.808126
Innate_Immune_System                            0.785249
Adaptive_Immune_System                          0.764829
Metabolism_of_lipids                            0.736997
Cytokine_Signaling_in_Immune_system             0.722466
Membrane_Trafficking                            0.711793
Intracellular_signaling_by_second_messengers    0.698741
Signaling_by_GPCR                               0.690652
Axon_guidance                                   0.663145
Signaling_by_Receptor_Tyrosine_Kinases          0.658621
Signaling_by_WNT                                0.631937
Signaling_by_NOTCH                              0.630210
Cell_Cycle_Checkpoints                          0.600696
Infectious_disease                              0.600141
Diseases_of_signal_transduction

In [17]:
tmp[tmp_corr.head(20).index].corr().round(3)

,Post_translational_protein_modification,RNA_Polymerase_II_Transcription,Cellular_responses_to_stress,Signaling_by_Rho_GTPases,Innate_Immune_System,Adaptive_Immune_System,Metabolism_of_lipids,Cytokine_Signaling_in_Immune_system,Membrane_Trafficking,Intracellular_signaling_by_second_messengers,Signaling_by_GPCR,Axon_guidance,Signaling_by_Receptor_Tyrosine_Kinases,Signaling_by_WNT,Signaling_by_NOTCH,Cell_Cycle_Checkpoints,Infectious_disease,Diseases_of_signal_transduction,Death_Receptor_Signalling,Metabolism_of_amino_acids_and_derivatives
Post_translational_protein_modification,1.000,0.912,0.849,0.833,-0.725,0.691,-0.637,-0.606,-0.676,0.631,0.610,-0.589,0.548,-0.529,-0.554,-0.548,0.520,-0.518,0.474,0.477
RNA_Polymerase_II_Transcription,0.912,1.000,0.865,0.835,-0.630,0.663,-0.606,-0.562,-0.552,0.654,0.543,-0.553,0.525,-0.532,-0.562,-0.511,0.448,-0.548,0.427,0.446
Cellular_responses_to_stress,0.849,0.865,1.000,0.926,-0.496,0.489,-0.421,-0.454,-0.407,0.493,0.452,-0.383,0.365,-0.403,-0.479,-0.481,0.343,-0.343,0.359,0.385
Signaling_by_Rho_GTPases,0.833,0.835,0.926,1.000,-0.527,0.485,-0.455,-0.451,-0.458,0.422,0.518,-0.407,0.421,-0.429,-0.405,-0.339,0.394,-0.324,0.509,0.376
Innate_Immune_System,-0.725,-0.630,-0.496,-0.527,1.000,-0.685,0.611,0.595,0.753,-0.493,-0.627,0.714,-0.657,0.620,0.510,0.451,-0.614,0.510,-0.560,-0.575
Adaptive_Immune_System,0.691,0.663,0.489,0.485,-0.685,1.000,-0.614,-0.572,-0.665,0.672,0.621,-0.605,0.593,-0.576,-0.487,-0.442,0.557,-0.498,0.551,0.505
Metabolism_of_lipids,-0.637,-0.606,-0.421,-0.455,0.611,-0.614,1.000,0.613,0.624,-0.563,-0.600,0.582,-0.557,0.496,0.413,0.427,-0.495,0.464,-0.453,-0.439
Cytokine_Signaling_in_Immune_system,-0.606,-0.562,-0.454,-0.451,0.595,-0.572,0.613,1.000,0.585,-0.542,-0.585,0.502,-0.481,0.471,0.572,0.594,-0.528,0.384,-0.531,-0.377
Membrane_Trafficking,-0.676,-0.552,-0.407,-0.458,0.753,-0.665,0.624,0.585,1.000,-0.419,-0.631,0.685,-0.618,0.538,0.414,0.418,-0.665,0.418,-0.526,-0.565
Intracellular_signaling_by_second_messengers,0.631,0.654,0.493,0.422,-0.493,0.672,-0.563,-0.542,-0.419,1.000,0.442,-0.435,0.515,-0.472,-0.600,-0.657,0.370,-0.487,0.391,0.351


In [18]:
# Dataset
X_for_pysr = tmp[tmp_corr.head(20).index]
y_for_pysr = data_for_pysr[3]

default_pysr_params = dict(
    populations=30,
    procs=4,
    model_selection="best",
)


In [19]:
# Learn equations
model = pysr.PySRRegressor(
        niterations=30,
        binary_operators=["plus", "sub", "mult", "div", "greater"],
        unary_operators=["neg", "square"],
        **default_pysr_params)

model.fit(X_for_pysr, y_for_pysr)

/mnt/home/ttesileanu/miniconda3/envs/cancerenv/lib/python3.9/site-packages/pysr/sr.py:1068: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/mnt/home/ttesileanu/miniconda3/envs/cancerenv/lib/python3.9/site-packages/pysr/sr.py:1581: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. Consider using feature selection techniques to select the most important features (you can do this automatically with the `select_k_features` parameter), or, alternatively, doing a dimensionality reduction beforehand. For example, `X = PCA(n_components=6).fit_transform(X)`, using scikit-learn's `PCA` class, will reduce the number of features to 6 in an interpretable way, as each resultant feature will be a linear combination of the original features. 
  warnings.warn(
  Activating project at `~/miniconda3/envs/cancerenv/share/julia/environments/pysr-0.

Started!

Cycles per second: 4.910e+04
Head worker occupation: 24.8%
Progress: 141 / 900 total iterations (15.667%)
Hall of Fame:
-----------------------------------------
Complexity  loss       Score     Equation
1           7.882e+00  4.041e-01  Post_translational_protein_modification
3           2.820e+00  5.139e-01  (Post_translational_protein_modification / 0.33718193)
5           1.875e+00  2.040e-01  (-1.2696389 + (RNA_Polymerase_II_Transcription * 2.6410763))
7           8.774e-01  3.798e-01  ((-1.2696389 - Cytokine_Signaling_in_Immune_system) + (RNA_Polymerase_II_Transcription * 2.6410763))
9           5.410e-01  2.418e-01  ((((RNA_Polymerase_II_Transcription * 1.6755315) + -1.1881642) + Post_translational_protein_modification) - Cytokine_Signaling_in_Immune_system)
11          4.659e-01  7.473e-02  (RNA_Polymerase_II_Transcription + (((Post_translational_protein_modification - Cytokine_Signaling_in_Immune_system) * 0.8295019) - (1.2611 - Post_translational_protein_modificatio

PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000            Post_translational_protein_modification   
	1   >>>>  0.530759  (Post_translational_protein_modification * 3.2...   
	2         0.447252  ((Post_translational_protein_modification * 3....   
	3         0.200976  (((Post_translational_protein_modification * 2...   
	4         0.205185  (((RNA_Polymerase_II_Transcription + -1.269637...   
	5         0.032773  (((RNA_Polymerase_II_Transcription + -1.269637...   
	6         0.200398  ((((((1.995275 - Signaling_by_Receptor_Tyrosin...   
	7         0.116630  (((((((Metabolism_of_lipids + Innate_Immune_Sy...   
	8         0.003205  (((((Cytokine_Signaling_in_Immune_system - ((C...   
	9         0.167937  ((((((greater(Innate_Immune_System, Signaling_...   
	10        0.001333  ((((((greater(Innate_Immune_System, Signaling_...   
	
	        loss  complexity  
	0   7.882146           1  
	1   2.726671           3  
	2   1.114691           5  
	3   0.745742           7  
	4   0.494729           9  
	5   0.463341          11  
	6   0.310340          13  
	7   0.245773          15  
	8   0.243421          18  
	9   0.205790          19  
	10  0.205516          20  
]

In [20]:
model.equations_.sort_values("score", ascending=False)[['complexity', 'loss', 'score', 'equation']]

,complexity,loss,score,equation
1,3,2.726671,0.530759,(Post_translational_protein_modification * 3.2...
2,5,1.114691,0.447252,((Post_translational_protein_modification * 3....
4,9,0.494729,0.205185,(((RNA_Polymerase_II_Transcription + -1.269637...
3,7,0.745742,0.200976,(((Post_translational_protein_modification * 2...
6,13,0.310340,0.200398,((((((1.995275 - Signaling_by_Receptor_Tyrosin...
9,19,0.205790,0.167937,"((((((greater(Innate_Immune_System, Signaling_..."
7,15,0.245773,0.116630,(((((((Metabolism_of_lipids + Innate_Immune_Sy...
5,11,0.463341,0.032773,(((RNA_Polymerase_II_Transcription + -1.269637...
8,18,0.243421,0.003205,(((((Cytokine_Signaling_in_Immune_system - ((C...
10,20,0.205516,0.001333,"((((((greater(Innate_Immune_System, Signaling_..."


In [21]:
model.sympy(9)

0.484470914643298*Cellular_responses_to_stress - 0.484470914643298*Cytokine_Signaling_in_Immune_system - 0.484470914643298*Metabolism_of_lipids + Post_translational_protein_modification + RNA_Polymerase_II_Transcription + 0.484470914643298*Signaling_by_Receptor_Tyrosine_Kinases - 0.484470914643298*greater(Innate_Immune_System, Signaling_by_GPCR) - 1.0

In [22]:
model.sympy()

3.2720592*Post_translational_protein_modification